# ✏️ Esercizi

## Preparazione del Notebook

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns

In [3]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

### Il problema delle due urne

Supponiamo che vi siano due urne.

- L'urna 1 ($U_1$) contiene 30 palline bianche (B) e 10 palline nere (N).
- L'urna 2 ($U_2$) contiene 20 palline bianche e 20 palline nere.

Supponiamo di scegliere una delle urne a caso e, senza guardare, di scegliere una pallina a caso. Se la pallina è bianca, qual è la probabilità che provenga dall'urna 1?

Quello che vogliamo è la probabilità condizionata che abbiamo scelto dall'Urna 1 dato che abbiamo ottenuto una pallina bianca, $P(U_1 \mid B)$.

Il problema ci fornisce le seguenti informazioni:

- $P(B \mid U_1)$ = 3/4,
- $P(B \mid U_2)$ = 1/2.

Il teorema di Bayes ci dice come le informazioni a disposizione si possono mettere in relazione con la domanda del problema:

$$
P(U_1 \mid B) = \frac{P(B \mid U_1) P(U_1)}{P(B)}
$$

Per calcolare la probabilità $P(B)$ usiamo il teorema della probabilità totale:

$$
P(B) = P(B \mid U_1) P(U_1) + P(B \mid U_2) P(U_2),
$$

ovvero

$$
P(B) = 3/4 \cdot 1/2 + 1/2 \cdot 1/2 = 5/8.
$$

Concludiamo applicando il teorema di Bayes:

$$
P(U_1 \mid B) = \frac{3/4 \cdot 1/2}{5/8} = 3/5.
$$


Il processo di aggiornamento bayesiano può essere anche svolto nel modo seguente. Riscrivo il teorema di Bayes nel modo seguente:

$$
P(H \mid D) = \frac{P(D \mid H) P(H)}{P(D)}
$$

La probabilità $P(H)$ è la probabilità delle ipotesi prima di avere osservato i dati. Nel nostro caso, le due ipotesi sono "Urna 1" e "Urna 2", entrambe con la stessa probabilità, dato che non abbiamo ragioni a priori per dare più peso ad un'ipotesi rispetto all'altra. Costruiamo una tabella con un DataFrame in cui inseriamo la colonna `prior`:

In [21]:
table = pd.DataFrame(index=["Urn 1", "Urn 2"])
table["prior"] = 1 / 2, 1 / 2
table

,prior
Urn 1,0.5
Urn 2,0.5


La probabilità $P(D \mid H)$ è la probabilità dei dati, data l'ipotesi. È chiamata verosimiglianza. La probabilità di una pallina bianca dato che viene estratta dall'Urna 1 è 3/4. La probabilità di una pallina bianca dato che viene estratta dall'Urna 1 è 1/2. Aggiungo alla tabella la colonna `likelihood`.

In [22]:
table["likelihood"] = 3 / 4, 1 / 2
table

,prior,likelihood
Urn 1,0.5,0.75
Urn 2,0.5,0.50


La probabilità $P(H \mid D)$ è la probabilità dell'ipotesi dopo avere osservato i dati. Si ottiene come il prodotto della verosimiglianza per la probabilità a priori, diviso per una *costante di normalizzazione*. Iniziamo a calcolare il la distribuzione a posteriori non normalizzata.

In [23]:
table["unnorm"] = table["prior"] * table["likelihood"]
table

,prior,likelihood,unnorm
Urn 1,0.5,0.75,0.375
Urn 2,0.5,0.50,0.250


La probabilità dei dati, $P(D)$, ovvero il numeratore bayesiano, è dato dalla somma di tutti i valori della distribuzione a posteriori non normalizzata:

In [24]:
prob_data = table["unnorm"].sum()

Possiamo ora normalizzare la distribuzione a posteriori:

In [25]:
table["posterior"] = table["unnorm"] / prob_data
table

,prior,likelihood,unnorm,posterior
Urn 1,0.5,0.75,0.375,0.6
Urn 2,0.5,0.50,0.250,0.4


### Il problema dei dadi

Il metodo precedente può anche essere usato quando ci sono più di due ipotesi. {cite:t}`downey2021think` discute il seguente problema. Supponiamo che nell'Urna 1 ci sia un dado a 6 facce, nell'Urna 2 un dado a 8 facce e nell'Urna 3 un dado a 12 facce. Un dado viene estratto a caso da un'urna e produce il risultato 1. Qual è la probabilità che ho usato un dado a 6 facce?

Inizio a definire le tre ipotesi.

In [26]:
table2 = pd.DataFrame(index=[6, 8, 12])
table2

""
6
8
12


Per evitare arrotondamenti uso la funzione `Fraction()`. Inizio a definire la distribuzione a priori.

In [27]:
from fractions import Fraction

table2["prior"] = Fraction(1, 3)
table2

,prior
6,1/3
8,1/3
12,1/3


Definisco la verosimiglianza. Se il dado è a 6 facce, la probabilità di ottenere 1 è 1/6; se il dado ha 8 facce è 1/8; se il dado ha 12 facce è 1/12.

In [28]:
table2["likelihood"] = Fraction(1, 6), Fraction(1, 8), Fraction(1, 12)
table2

,prior,likelihood
6,1/3,1/6
8,1/3,1/8
12,1/3,1/12


Trovo la distribuzione a posteriori non normalizzata.

In [29]:
table2["unnorm"] = table2["prior"] * table2["likelihood"]
table2

,prior,likelihood,unnorm
6,1/3,1/6,1/18
8,1/3,1/8,1/24
12,1/3,1/12,1/36


Normalizzo.

In [30]:
prob_data = table2["unnorm"].sum()
table2["posterior"] = table2["unnorm"] / prob_data
table2

,prior,likelihood,unnorm,posterior
6,1/3,1/6,1/18,4/9
8,1/3,1/8,1/24,1/3
12,1/3,1/12,1/36,2/9


La probabilità posteriore del dado a 6 facce è 4/9, che è più grande delle probabilità degli altri dadi, 3/9 e 2/9. Intuitivamente, il dado a 6 facce è il più probabile perché ha la probabilità più grande di produrre il risultato che abbiamo osservato.

### L'errore dell'avvocato difensore

Una donna è stata assassinata e il marito è sotto processo per questo crimine. Durante il processo emerge che l'imputato aveva un passato di abusi sulla moglie. L'avvocato difensore sostiene che questa prova dovrebbe essere esclusa perché irrilevante, dato che solo 1 uomo su 10000 che abusa della moglie finisce poi per ucciderla. Dovrebbe il giudice accettare la richiesta dell'avvocato difensore di escludere questa prova?

Supponiamo che la cifra 1 su 10000 fornita dall'avvocato difensore sia corretta e assumiamo inoltre le seguenti probabilità per una popolazione rilevante di mariti e mogli: 1 marito su 10 abusa della moglie, 1 moglie su 5 assassinata viene uccisa dal marito, e il 50% dei mariti che uccidono le loro mogli le aveva precedentemente abusate.

Definiamo $ A $ come l'evento in cui il marito abusa della moglie, e $ G $ come l'evento in cui il marito è colpevole. L'argomento della difesa è che $ P(G \mid A) = \frac{1}{10000} $, quindi la colpevolezza è estremamente improbabile condizionata a una storia pregressa di abusi.

Tuttavia, l'avvocato difensore omette di considerare un fatto cruciale: in questo caso, sappiamo che la moglie è stata assassinata. Pertanto, la probabilità rilevante non è $ P(G \mid A) $, ma $ P(G \mid A,M) $, dove $ M $ è l'evento in cui la moglie è stata assassinata.

Applicando la regola di Bayes con un ulteriore fattore di condizionamento, otteniamo:

$$
P(G \mid A,M) = \frac{P(A \mid G,M)P(G \mid M)}{P(A \mid G,M)P(G \mid M) + P(A \mid G^c,M)P(G^c \mid M)} 
= \frac{0.5 \cdot 0.2}{0.5 \cdot 0.2 + 0.1 \cdot 0.8} 
= \frac{59}{100}.
$$

La condizionata della Regola di Bayes e include le seguenti componenti:

- $ P(G \mid A,M) $: La probabilità che il marito sia colpevole, dato che ha abusato della moglie e che questa è stata assassinata.
- $ P(A \mid G,M) $: La probabilità che il marito abbia abusato della moglie, dato che è colpevole e che la moglie è stata assassinata. Questa è stata assunta come $0.5$ nel problema.
- $ P(G \mid M) $: La probabilità che il marito sia colpevole, dato che la moglie è stata assassinata. Questa è stata assunta come $0.2$ nel problema.
- $ P(A \mid G^c,M) $: La probabilità che il marito abbia abusato della moglie, dato che non è colpevole ma la moglie è stata assassinata. Questa è stata assunta come $0.1$ nel problema.
- $ P(G^c \mid M) $: La probabilità che il marito non sia colpevole, dato che la moglie è stata assassinata. Questa è semplicemente $1 - P(G \mid M) = 0.8$.

La formula permette di combinare diverse fonti di informazione per arrivare a una stima più accurata della probabilità di colpevolezza. In questo caso, la probabilità a posteriori di colpevolezza, $ P(G \mid A,M) $, si rivela essere molto più alta della probabilità iniziale $ P(G \mid A) $ proposta dall'avvocato difensore. Pertanto, contrariamente all'argomento dell'avvocato, la storia di abusi del marito è un'informazione molto importante che non dovrebbe essere ignorata nel processo. Condizionando sulla prova degli abusi, la probabilità di colpevolezza sale da $ P(G \mid M) = 0.2 $ a $ P(G \mid A,M) \approx 0.59 $, contraddicendo quindi l'argomento dell'avvocato difensore.

Nel calcolo di $ P(G \mid A,M) $, non abbiamo utilizzato la probabilità $ P(G\mid A) $ fornita dall'avvocato difensore; è irrilevante per il nostro calcolo perché non tiene conto del fatto che la moglie è stata assassinata. 

Questo esempio mette in evidenza come sia necessario  condizionare su tutte le prove disponibili. Inoltre, questo esempio mette in luce le complessità e le potenziali insidie nell'utilizzo della probabilità condizionata, soprattutto quando sono in gioco variabili di confondimento.

## Watermark

In [14]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Sat Mar 16 2024

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

seaborn   : 0.13.2
pandas    : 2.2.1
matplotlib: 3.8.3
numpy     : 1.26.4
arviz     : 0.17.0

Watermark: 2.4.3

